## Altana A.I. Takehome Assignment 
### Candidate for Hire: James (Jay) Soley
#### July 29, 2021 

In [69]:
# Import Numpy and Pandas Libraries and Load in CSV Data

import numpy as np
import pandas as pd
import seaborn as sb
pd.set_option('display.max_rows', None)

train_df = pd.read_csv('/Users/jamessoley/Documents/altana/ds-project-train.csv')

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [70]:
train_df.describe()

,Unnamed: 0,WEIGHT..LB.,WEIGHT..KG.,QUANTITY,MEASUREMENT,CONTAINER.COUNT
count,143280.000000,1.316930e+05,1.316930e+05,1.316930e+05,131693.000000,131693.000000
mean,95604.264391,3.868200e+04,1.758005e+04,1.519561e+03,79.271412,1.564381
std,55174.831229,1.155783e+06,5.251678e+05,7.820183e+04,1409.380657,2.070290
min,1.000000,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000
25%,47799.750000,1.357400e+03,6.170000e+02,1.460000e+02,1.000000,1.000000
50%,95690.500000,6.010400e+03,2.732000e+03,4.650000e+02,10.000000,1.000000
75%,143374.500000,2.398600e+04,1.090270e+04,1.200000e+03,48.000000,1.000000
max,191040.000000,1.331000e+08,6.050000e+07,2.833283e+07,431431.000000,127.000000


### Investigate the Class Labels

In [71]:
train_pivot = train_df.groupby('COUNTRY.OF.ORIGIN')['COUNTRY.OF.ORIGIN'].count().sort_values(ascending=False).to_frame('country_count').reset_index()
train_pivot

,COUNTRY.OF.ORIGIN,country_count
0,China,94041
1,Hong Kong,4934
2,South Korea,4271
3,Singapore,4147
4,Oman,2345
5,Panama,2298
6,China Taiwan,2192
7,Malaysia,2061
8,Guatemala,1659
9,Spain,1374


#### Clearly we see the the balance of the classes is very imbalanced with China representing ~ 70% of the overall data

#### Investigate if Labels and Features contain missing data

In [72]:
### Only select the Labels and Features
train_df[['COUNTRY.OF.ORIGIN','ARRIVAL.DATE','WEIGHT..KG.','US.PORT','PRODUCT.DETAILS']].isnull().sum()

COUNTRY.OF.ORIGIN    12795
ARRIVAL.DATE         11587
WEIGHT..KG.          11587
US.PORT              12093
PRODUCT.DETAILS      11662
dtype: int64

#### We're missing 12.7K Class Labels according to above -- let's take a look at what some of these rows look like

In [73]:
### Let's Take a Look at the Dataset so we can better see what's actually happening with the NULL Values
bool_series = pd.isnull(train_df['COUNTRY.OF.ORIGIN'])
#train_df[bool_series]

#### It's unclear to me why we have data with missing class labels but I don't think it makes very much sense to train on this data and I don't see any pattern which can allow me to impute the class labels from domain knowledge of the dataset. For example, I thought "Carrier.City" might be related to Country.Of.Origin but it doesn't appear that way. Therefore, I think the best answer is remove observations where we don't have the class label

In [74]:
### Remove Rows with missing class label from the dataset
train_df_labels = train_df[bool_series == False]
train_df_labels.isnull().sum()

Unnamed: 0                 0
SHIPPER                 9278
SHIPPER.ADDRESS        99714
CONSIGNEE                  0
CONSIGNEE.ADDRESS      97922
ZIPCODE                10613
NOTIFY                 14586
NOTIFY.ADDRESS         17043
BILL.OF.LADING             0
ARRIVAL.DATE               0
WEIGHT..LB.                0
WEIGHT..KG.                0
US.PORT                    0
QUANTITY                   0
Q.UNIT                   768
MEASUREMENT                0
M.UNIT                 25229
SHIP.REGISTERED.IN         8
VESSEL.NAME                0
CONTAINER.COUNT            0
PRODUCT.DETAILS           75
MARKS.AND.NUMBERS       1205
COUNTRY.OF.ORIGIN          0
DISTRIBUTION.PORT     101597
CARRIER.CODE               0
CARRIER.NAME             296
CARRIER.ADDRESS         2604
CARRIER.CITY            5603
CARRIER.STATE          60805
CARRIER.ZIP            28464
dtype: int64

#### It appears that by removing the rows with the missing class labels that we've also solved our issues with the missing values in the features that we're interested in using to predict the Country.Of.Origin: US.Port, WEIGHT..KG., ARRIVAL.DATE, PRODUCT.DETAILS (is missing a small amount so nbd)

### Feature Engineering
#### Let's take a closer look at how some of the features look so we can devise a strategy of attack
##### Plan: B/c I'm heading towards XGBoost to fit my classifier (a tree based model) I don't plan to change the Weight.KG variable as it doesn't need feature engineering. I do plan on turning port into categoricals using one hot encoding and the temporal feature also needs to be coded in a way that XGBoost can understand

In [75]:
### Weight KG -- Cleaned Name

weight_kg = train_df_labels['WEIGHT..KG.']

### US.PORT
### One Hot Encoding
port_one_hot = train_df_labels['US.PORT']
port_dummies = pd.get_dummies(port_one_hot)
print(port_one_hot.drop_duplicates())

0                                   Long Beach, California
6                                    Jacksonville, Florida
7                                        Norfolk, Virginia
47                                       Savannah, Georgia
53                              Charleston, South Carolina
64                                     Seattle, Washington
169                                Los Angeles, California
412               New York/Newark Area, Newark, New Jersey
413                                         Houston, Texas
1105                                       Mobile, Alabama
1158                                    Tacoma, Washington
1468                            Philadelphia, Pennsylvania
1970                                 San Juan, Puerto Rico
2162                                    New York, New York
5175                                New Orleans, Louisiana
6224                                   Oakland, California
9278                              Port Everglades, Flori

#### Note: I'm not entirely sure why BOSTON, OAKLAND, NEW YORK, CHARLESTON are different than their counterparts but I suppose it's possible that a Capitalized place is different? Also don't know why Vancouver is included as a US Port. I chose not to remove these observations b/c I have no domain knowledge and instances seem to be rare of the dataset but it might be worth further investigation

In [76]:
### ARRIVAL.DATE
### Create Variables to try and Capture the Driving Factors

### Ensure that ARRIVAL.DATE is actually a datetime
train_df_labels['my_dates'] = pd.to_datetime(train_df_labels['ARRIVAL.DATE'])

### DOW
train_df_labels['day_of_week'] = train_df_labels['my_dates'].dt.day_name()

### Weekend v. Weekday (couldn't find native function in Pandas)
def weekend(row):
    #Project
    if row['day_of_week'] == 'Saturday' or row['day_of_week'] == 'Sunday':
        return 'Weekend'
    else:
        return 'Weekday'

train_df_labels['week_type'] = train_df_labels.apply(weekend, axis=1)

### Turn dow and week_type into Dummies
dow_dummies = pd.get_dummies(train_df_labels['day_of_week'])
weektype_dummies = pd.get_dummies(train_df_labels['week_type'])

### Time of Year
train_df_labels['month'] = train_df_labels['my_dates'].dt.month_name()

### Season
def season(row):
    #Project
    if row['month'] == 12 or row['month'] == 1 or row['month'] == 2:
        return 'Winter'
    elif row['month'] == 3 or row['month'] == 4 or row['month'] == 5:
        return 'Spring'
    elif row['month'] == 6 or row['month'] == 7 or row['month'] == 8:
        return 'Summer'
    else:
        return 'Fall'

train_df_labels['season'] = train_df_labels.apply(season, axis=1)

### Turn dow and week_type into Dummies
month_dummies = pd.get_dummies(train_df_labels['month'])
season_dummies = pd.get_dummies(train_df_labels['season'])

train_df_labels.head()

<ipython-input-76-d20a5cb8b734>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['my_dates'] = pd.to_datetime(train_df_labels['ARRIVAL.DATE'])
<ipython-input-76-d20a5cb8b734>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['day_of_week'] = train_df_labels['my_dates'].dt.day_name()
<ipython-input-76-d20a5cb8b734>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,Unnamed: 0,SHIPPER,SHIPPER.ADDRESS,CONSIGNEE,CONSIGNEE.ADDRESS,ZIPCODE,NOTIFY,NOTIFY.ADDRESS,BILL.OF.LADING,ARRIVAL.DATE,...,CARRIER.NAME,CARRIER.ADDRESS,CARRIER.CITY,CARRIER.STATE,CARRIER.ZIP,my_dates,day_of_week,week_type,month,season
0,1,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359136,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
1,3,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359139,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
2,6,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359147,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
3,7,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359149,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
4,8,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359150,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall


#### Note: I looked into a smarter way to do a date or time transformation and thought very seriously about this: https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca . What ultimately stopped me though is the fact that I'm using XGBoost and creating two variables as opposed to one can be problematic for this algo b/c "they will fail to process these two features simultaneously whereas the cos/sin values are expected to be considered as one single coordinates system."

### Build Train Test off of Training Data Provided
#### Leave Validation Dataset Given by Altana for Final Model Selection

In [77]:
import numpy as np
from sklearn.model_selection import train_test_split

### Combine all features Together into one training dataset
### Class Labels Need to Be Numeric
train_df_labels['COUNTRY.OF.ORIGIN'] = train_df_labels['COUNTRY.OF.ORIGIN'].astype('category')
train_df_labels['COUNTRY.OF.ORIGIN_CAT'] = train_df_labels['COUNTRY.OF.ORIGIN'].cat.codes

### Xs are Featured Engineered Above and y is Numeric Category Codes from Country 
y = train_df_labels['COUNTRY.OF.ORIGIN_CAT']
X = pd.concat([weight_kg,port_dummies,dow_dummies,weektype_dummies,month_dummies,season_dummies], axis=1)

### Create Training and Testing Sets 2/3rds (train) v. 1/3 (holdout)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train.describe()

<ipython-input-77-32af7bdaacc3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['COUNTRY.OF.ORIGIN'] = train_df_labels['COUNTRY.OF.ORIGIN'].astype('category')
<ipython-input-77-32af7bdaacc3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['COUNTRY.OF.ORIGIN_CAT'] = train_df_labels['COUNTRY.OF.ORIGIN'].cat.codes


,WEIGHT..KG.,"Atlanta, Georgia",BOSTON,"Baltimore, Maryland","Boston, Massachusetts","Brunswick, Georgia",CHARLESTON,"Carquinez Strait, California","Charleston, South Carolina","Chester, Pennsylvania",...,February,January,July,June,March,May,November,October,September,Fall
count,8.742400e+04,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.0,87424.000000,87424.000000,87424.000000,...,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.000000,87424.0
mean,1.891680e+04,0.000092,0.000011,0.001281,0.000698,0.000023,0.0,0.000011,0.017695,0.000160,...,0.123353,0.104800,0.079635,0.029877,0.078617,0.057135,0.079658,0.120836,0.069089,1.0
std,5.837378e+05,0.009566,0.003382,0.035770,0.026406,0.004783,0.0,0.003382,0.131842,0.012654,...,0.328844,0.306297,0.270729,0.170250,0.269141,0.232102,0.270764,0.325939,0.253606,0.0
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,6.190000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,2.765950e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.096025e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,6.050000e+07,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


#### Fit XGBoost Classifier -- Model 1

In [10]:
### No Rebalancing of the Classes and Keeping all Classes

#Import xgboost as xgb
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:57:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 72.05%


#### I was a bit worried about runtime but it wasn't too bad -- about 7 minutes run locally on my personal Mac

In [12]:
from sklearn.metrics import classification_report

# df1 = train_df_labels.sort_values('COUNTRY.OF.ORIGIN_CAT', ascending=True).drop_duplicates('COUNTRY.OF.ORIGIN').sort_index()
# country_key = df1[['COUNTRY.OF.ORIGIN_CAT','COUNTRY.OF.ORIGIN']].sort_values('COUNTRY.OF.ORIGIN_CAT', ascending=True)
# country_key_values = country_key['COUNTRY.OF.ORIGIN'].tolist()
# len(country_key_values)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.86      0.92        81
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00         3
           3       0.56      0.31      0.40        78
           4       0.21      0.06      0.10        62
           6       0.00      0.00      0.00         2
           7       0.47      0.29      0.36        89
           8       0.00      0.00      0.00         1
           9       0.50      0.38      0.43        37
          10       0.00      0.00      0.00         1
          11       0.79      0.81      0.80       254
          12       0.82      0.88      0.85     31203
          13       0.16      0.14      0.15       727
          14       0.77      0.30      0.43        33
          15       0.99      0.98      0.99       413
          17       0.44      0.16      0.23        45
          18       0.00      0.00      0.00         2
          19       0.08    

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### In analyzing the above classification report I think there are a few takeaways:
##### 1) XGBoost does a decent job training and predicting with a relatively simple set of simple features 
##### 2) .72 accuracy is a nice place to start and will serve as a baseline to compare to future models and improvements
##### 3) We need to trim out countries with very low data as it's going to be difficult to predict with this information. While it's somewhat arbitrary I think combining countries with less than 10 observations is a good place to start. In 1/3 vs 2/3rd train test scenario the training set will likely be too small to actually learn differences b/w classes
##### 4) I think a SMOTE procedure or some hyperparameter tuning around the XGBoost class weights will suit this data well given the dominance of China in the data

In [78]:
#### Trimming Down the Classes to Less than or Equal to 10 Observations

### Country_Other
def country_other(row):
    #Project
    if row['COUNTRY.OF.ORIGIN'] in ['Guadeloupe','Kuwait','Denmark','Cayman Isl','Guam','Belize',
                                    'Philippines','Federal Republic of Germany','US Virgin Is','Taiwan',
                                    'Neth Antilles','Cambodia','Saudi Arabia','Venezuela','Malta','Iceland','Ecuador',
                                    'New Zealand','Bermuda','Indonesia','Morocco','Trinidad','Peru']:
        return 'Other'
    else:
        return row['COUNTRY.OF.ORIGIN']

train_df_labels['COUNTRY.OF.ORIGIN_TRIM'] = train_df_labels.apply(country_other, axis=1)

train_df_labels.head()

<ipython-input-78-3f9adcf8878d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['COUNTRY.OF.ORIGIN_TRIM'] = train_df_labels.apply(country_other, axis=1)


,Unnamed: 0,SHIPPER,SHIPPER.ADDRESS,CONSIGNEE,CONSIGNEE.ADDRESS,ZIPCODE,NOTIFY,NOTIFY.ADDRESS,BILL.OF.LADING,ARRIVAL.DATE,...,CARRIER.CITY,CARRIER.STATE,CARRIER.ZIP,my_dates,day_of_week,week_type,month,season,COUNTRY.OF.ORIGIN_CAT,COUNTRY.OF.ORIGIN_TRIM
0,1,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359136,09/20/2012,...,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,12,China
1,3,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359139,09/20/2012,...,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,12,China
2,6,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359147,09/20/2012,...,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,12,China
3,7,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359149,09/20/2012,...,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,12,China
4,8,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359150,09/20/2012,...,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,12,China


### Refit Data with fewer classes -- Model 2

In [79]:
### Combine all features Together into one training dataset

### Class Labels Need to Be Numeric Based on Updated Information _TRIM VAR
train_df_labels['COUNTRY.OF.ORIGIN_TRIM'] = train_df_labels['COUNTRY.OF.ORIGIN_TRIM'].astype('category')
train_df_labels['COUNTRY.OF.ORIGIN_CAT2'] = train_df_labels['COUNTRY.OF.ORIGIN_TRIM'].cat.codes

### Xs are Featured Engineered Above and y is Numeric Category Codes from Country 
y = train_df_labels['COUNTRY.OF.ORIGIN_CAT2']
X = pd.concat([weight_kg,port_dummies,dow_dummies,weektype_dummies,month_dummies,season_dummies], axis=1)

### Create Training and Testing Sets 2/3rds (train) v. 1/3 (holdout)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

<ipython-input-79-e38112ce42ca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['COUNTRY.OF.ORIGIN_TRIM'] = train_df_labels['COUNTRY.OF.ORIGIN_TRIM'].astype('category')
<ipython-input-79-e38112ce42ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_labels['COUNTRY.OF.ORIGIN_CAT2'] = train_df_labels['COUNTRY.OF.ORIGIN_TRIM'].cat.codes


In [56]:
model = XGBClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:12:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 81.50%


In [62]:
#from sklearn.metrics import classification_report

#Get the Labels in the Report
df1 = train_df_labels.sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True).drop_duplicates('COUNTRY.OF.ORIGIN_TRIM').sort_index()
country_key = df1[['COUNTRY.OF.ORIGIN_CAT2','COUNTRY.OF.ORIGIN_TRIM']].sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True)
country_key_values = country_key['COUNTRY.OF.ORIGIN_TRIM'].tolist()
#len(country_key_values)

print(classification_report(y_test, y_pred, target_names=country_key_values))

                    precision    recall  f1-score   support

    American Samoa       0.94      0.84      0.89        81
         Argentina       0.60      0.21      0.32        14
         Australia       0.00      0.00      0.00         3
           Bahamas       0.72      0.37      0.49        78
           Belgium       0.36      0.15      0.21        62
            Brazil       0.72      0.33      0.45        89
            Canada       0.75      0.49      0.59        37
             Chile       0.86      0.84      0.85       254
             China       0.85      0.98      0.91     31203
      China Taiwan       0.56      0.29      0.38       727
          Colombia       0.75      0.36      0.49        33
        Costa Rica       0.98      0.99      0.99       413
Dominican Republic       0.70      0.42      0.53        45
             Egypt       0.52      0.15      0.24        84
            France       0.50      0.28      0.36        18
           Germany       0.62      0.36

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Not suprisingly, when we take the countries that have less than or equal to 10 total observations and combine them into an "other" group our model performs much better overall. This is to be expected as we combined 23 countries together but it's still nice to see. Individual F1 Scores still leave something to be desired but we're improving. Some countries are wayyyy better than others (besides China). For example, Costa Rica is very accurate (F1 Score) while Italy isn't (F1 Score)

### Rebalancing the Training Dataset using SMOTE to try and oversample minority classes and hopefully lesson the China dominant effect. Model -- 3

In [80]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from matplotlib import pyplot

# transform the dataset
strategy = {0:1000,1:1000,2:1000,3:1000,4:1000,5:1000,6:1000,7:1000,8:10000,9:1000,10:1000,
            11:1000,12:1000,13:1000,14:1000,15:1000,16:1000,17:1000,18:1000,19:1000,20:1634,
            21:1000,22:1000,23:1000,24:1000,25:1000,26:1000,27:1000,28:1000,29:1000,30:1000,
            31:1000,32:1000,33:1000,34:1000,35:1000,36:1353,37:1000,38:1416,39:1000,40:1000,
            41:1000,42:1000,43:1000,44:1000,45:1000,46:1000,47:1000}
undersample = RandomUnderSampler(sampling_strategy=strategy)
X_train, y_train = undersample.fit_resample(X_train, y_train)  
# summarize distribution
counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

ValueError: With under-sampling methods, the number of samples in a class should be less or equal to the original number of samples. Originally, there is 171 samples and 1000 samples are asked.

#### I tried an oversample technique but I don't have the compute on my personal Mac to execute it properly. Therefore, I will try an undersampling technique instead.

In [27]:
#Import xgboost as xgb
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:02:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 78.31%


In [29]:
from sklearn.metrics import classification_report

#Get the Labels in the Report
df1 = train_df_labels.sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True).drop_duplicates('COUNTRY.OF.ORIGIN_TRIM').sort_index()
country_key = df1[['COUNTRY.OF.ORIGIN_CAT2','COUNTRY.OF.ORIGIN_TRIM']].sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True)
country_key_values = country_key['COUNTRY.OF.ORIGIN_TRIM'].tolist()
#len(country_key_values)

print(classification_report(y_test, y_pred, target_names=country_key_values))

                    precision    recall  f1-score   support

    American Samoa       0.83      0.89      0.86        81
         Argentina       0.16      0.36      0.22        14
         Australia       0.08      0.33      0.13         3
           Bahamas       0.32      0.35      0.33        78
           Belgium       0.13      0.18      0.15        62
            Brazil       0.42      0.34      0.37        89
            Canada       0.36      0.49      0.41        37
             Chile       0.80      0.81      0.81       254
             China       0.83      0.97      0.89     31203
      China Taiwan       0.40      0.12      0.19       727
          Colombia       0.31      0.39      0.35        33
        Costa Rica       0.97      0.99      0.98       413
Dominican Republic       0.46      0.64      0.54        45
             Egypt       0.24      0.33      0.28        84
            France       0.09      0.33      0.14        18
           Germany       0.42      0.25

#### Surprisingly, undersampling China and oversampling the other classes hurts the overall model. I do believe there is ALOT more work to be done in researching this but in the spirit on the "3 hour takehome" I'll stop and won't go down the deep rabbit hole of differnt reblancing strategies of the classes. I'm also disappointed my compute power can't fully balance all classes equally.

### Using NLP to Try and Extract Signal out of the Product Details Field

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 

### Expand the StopWords List by Eyeballing things that don't make sense
stop_words = text.ENGLISH_STOP_WORDS.union(['item','vendor','type','po','dept','description',
                                            'pls','cargo','assortment','container','contains',
                                            'material','materials','shipment','shipper','qty',
                                            'purchase','product','procedures','pcs','order',
                                            'destination','delivery','count','code','ct',
                                            'department','negotiable','negotiated','non',
                                            'number','packaging','supplier'])

corpus = train_df_labels['PRODUCT.DETAILS'].values.astype('U')
cv = CountVectorizer(max_features = 100, stop_words=stop_words)
product_details_parsed = cv.fit_transform(corpus)
print(cv.get_feature_names())

['00', '00007', '00014', '00016', '00018', '00025', '00063', '0020', '0023', '0040', '0042', '0043', '0073', '0078742000008', '10', '100', '12', '13', '16', '20', '2012', '2013', '24', '30', '40', '43', '48', '5oz', '9503', '9503000073', 'allowed', 'artificial', 'asst', 'black', 'bx', 'canada', 'carrier', 'cartons', 'cbm', 'certify', 'chair', 'collect', 'com', 'consistent', 'copy', 'cotton', 'ctn', 'ctns', 'cy', 'declaration', 'declare', 'details', 'elwood', 'expeditors', 'fg', 'flow', 'footwear', 'freight', 'fresh', 'furniture', 'gln', 'great', 'houston', 'hs', 'hts', 'issue', 'itemized', 'kgs', 'knit', 'ladies', 'load', 'ms', 'op', 'packing', 'pam', 'place', 'plastic', 'polyester', 'pono', 'regulated', 'release', 'savannah', 'sea', 'set', 'sk', 'solid', 'southgate', 'steel', 'stock', 'storage', 'style', 'suffolk', 'toys', 'value', 'vancouver', 'virtual', 'wal', 'way', 'white', 'wood']


#### It's hard to know that the right cutoff should be in terms of number of NLP bag of words features. We know XGBoost will guard against overfitting so it might just be a matter of compute

In [82]:
### Turn Product Details into a DF and 
df_nlp = pd.DataFrame(product_details_parsed.todense(), columns=cv.get_feature_names())

df_nlp.head()

,00,00007,00014,00016,00018,00025,00063,0020,0023,0040,...,style,suffolk,toys,value,vancouver,virtual,wal,way,white,wood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [83]:
### Update X to include the NLP Vectorized Features

X1 = pd.concat([weight_kg,port_dummies,dow_dummies,weektype_dummies,month_dummies,season_dummies], axis=1).reset_index()
X = pd.concat([X1,df_nlp], axis=1)

#Create Training and Testing Sets 2/3rds (train) v. 1/3 (holdout)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Fit Model Using the NLP Bag of Words Features with No Class Rebalancing -- Model 4

In [84]:
### Fit XGboost

#Import xgboost as xgb
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:00:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 92.96%


In [85]:
from sklearn.metrics import classification_report

#Get the Labels in the Report
df1 = train_df_labels.sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True).drop_duplicates('COUNTRY.OF.ORIGIN_TRIM').sort_index()
country_key = df1[['COUNTRY.OF.ORIGIN_CAT2','COUNTRY.OF.ORIGIN_TRIM']].sort_values('COUNTRY.OF.ORIGIN_CAT2', ascending=True)
country_key_values = country_key['COUNTRY.OF.ORIGIN_TRIM'].tolist()
#len(country_key_values)

print(classification_report(y_test, y_pred, target_names=country_key_values))

                    precision    recall  f1-score   support

    American Samoa       1.00      0.96      0.98        81
         Argentina       1.00      0.57      0.73        14
         Australia       1.00      0.33      0.50         3
           Bahamas       0.78      0.64      0.70        78
           Belgium       0.80      0.63      0.70        62
            Brazil       0.89      0.88      0.88        89
            Canada       0.88      0.59      0.71        37
             Chile       0.94      0.97      0.95       254
             China       0.95      0.99      0.97     31203
      China Taiwan       0.84      0.73      0.78       727
          Colombia       0.81      0.79      0.80        33
        Costa Rica       0.99      1.00      1.00       413
Dominican Republic       0.94      0.71      0.81        45
             Egypt       0.96      0.96      0.96        84
            France       0.78      0.39      0.52        18
           Germany       0.81      0.82

#### With even the most basic NLP (Bag of Words) the improvement in the model is significant our accuracy goes to .93% and the F1 scores for some smaller countries are significantly improved. I would imagine that more advanced approaches like tf idf, word to vec, BERT Library could push performance even higher.

### Read in Validation Model, Prepare Data and Predict (Using Trained Model 4)

In [49]:
### Read in Validation Data for Prediction
validation_df = pd.read_csv('/Users/jamessoley/Documents/altana/ds-project-validation.csv')

### Weight KG -- Cleaned Name
weight_kg_valid = validation_df['WEIGHT..KG.']

### US.PORT
### One Hot Encoding
port_one_hot_valid = validation_df['US.PORT']
port_dummies_valid = pd.get_dummies(port_one_hot_valid)
print(port_one_hot_valid.drop_duplicates())

### ARRIVAL.DATE
### Create Variables to try and Capture the Driving Factors

### Ensure that ARRIVAL.DATE is actually a datetime
validation_df['my_dates'] = pd.to_datetime(validation_df['ARRIVAL.DATE'])

### DOW
validation_df['day_of_week'] = validation_df['my_dates'].dt.day_name()
validation_df['week_type'] = validation_df.apply(weekend, axis=1)

### Turn dow and week_type into Dummies
dow_dummies_valid = pd.get_dummies(validation_df['day_of_week'])
weektype_dummies_valid = pd.get_dummies(validation_df['week_type'])

### Time of Year
validation_df['month'] = validation_df['my_dates'].dt.month_name()
validation_df['season'] = validation_df.apply(season, axis=1)

### Turn dow and week_type into Dummies
month_dummies_valid = pd.get_dummies(validation_df['month'])
season_dummies_valid = pd.get_dummies(validation_df['season'])

validation_df.head()

0                                   Long Beach, California
4                                    Jacksonville, Florida
5                                        Norfolk, Virginia
12                                       Savannah, Georgia
13                                          Houston, Texas
15                                     Seattle, Washington
52                                 Los Angeles, California
397                                     Tacoma, Washington
513                             Philadelphia, Pennsylvania
521               New York/Newark Area, Newark, New Jersey
559                             Charleston, South Carolina
670                                  San Juan, Puerto Rico
1732                                    New York, New York
3062                              Port Everglades, Florida
3100                                                   NaN
3119                                  Wilmington, Delaware
3152                                        Miami, Flori

,Unnamed: 0,SHIPPER,SHIPPER.ADDRESS,CONSIGNEE,CONSIGNEE.ADDRESS,ZIPCODE,NOTIFY,NOTIFY.ADDRESS,BILL.OF.LADING,ARRIVAL.DATE,...,CARRIER.NAME,CARRIER.ADDRESS,CARRIER.CITY,CARRIER.STATE,CARRIER.ZIP,my_dates,day_of_week,week_type,month,season
0,2,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359138,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
1,4,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359145,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
2,5,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359146,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
3,10,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359141,09/20/2012,...,COMPAGNIE MARITIME D-AFFRETEMENT,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall
4,11,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,EGLV143285966961,09/20/2012,...,EVERGREEN LINE,NO 163 SEC 1 HSIN-NAN ROAD LUCHU HSIAN,TAOYUAN HSIEN 338,NaN,NaN,2012-09-20,Thursday,Weekday,September,Fall


### Set up the Class Labels for y_valid

In [50]:
#### Trimming Down the Classes to Less than or Equal to 10 Observations

validation_df['COUNTRY.OF.ORIGIN_TRIM'] = train_df_labels.apply(country_other, axis=1)
validation_df.head()

,Unnamed: 0,SHIPPER,SHIPPER.ADDRESS,CONSIGNEE,CONSIGNEE.ADDRESS,ZIPCODE,NOTIFY,NOTIFY.ADDRESS,BILL.OF.LADING,ARRIVAL.DATE,...,CARRIER.ADDRESS,CARRIER.CITY,CARRIER.STATE,CARRIER.ZIP,my_dates,day_of_week,week_type,month,season,COUNTRY.OF.ORIGIN_TRIM
0,2,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359138,09/20/2012,...,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,China
1,4,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359145,09/20/2012,...,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,China
2,5,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359146,09/20/2012,...,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,China
3,10,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,CMDUSZ2359141,09/20/2012,...,5701 LAKE WRIGHT DR,NORFOLK,VA,23502,2012-09-20,Thursday,Weekday,September,Fall,China
4,11,NaN,NaN,-NOT AVAILABLE-,NaN,0,NaN,NaN,EGLV143285966961,09/20/2012,...,NO 163 SEC 1 HSIN-NAN ROAD LUCHU HSIAN,TAOYUAN HSIEN 338,NaN,NaN,2012-09-20,Thursday,Weekday,September,Fall,China


In [51]:
### Merge in Class Labels from train_df_labels on 'COUNTRY.OF.ORIGIN_TRIM' from training_df to validation_df
### Note: I know this is a bad way to do this but my pd.merge statements weren't working correctly (I'm sure I'm doing something wrong)

def class_labeler(row):
    if row['COUNTRY.OF.ORIGIN_TRIM'] == 'American Samoa':
        return(0)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Argentina':
        return(1)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Australia':
        return(2)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Bahamas':
        return(3)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Belgium':
        return(4)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Brazil':
        return(5)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Canada':
        return(6)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Chile':
        return(7)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'China':
        return(8)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'China Taiwan':
        return(9)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Colombia':
        return(10)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Costa Rica':
        return(11)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Dominican Republic':
        return(12)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Egypt':
        return(13)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'France':
        return(14)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Germany':
        return(15)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Greece':
        return(16)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Guatemala':
        return(17)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Haiti':
        return(18)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Honduras':
        return(19)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Hong Kong':
        return(20)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'India':
        return(21)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Israel':
        return(22)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Italy':
        return(23)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Jamaica':
        return(24)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Japan':
        return(25)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Jordan':
        return(26)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Malaysia':
        return(27)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Mexico':
        return(28)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Netherlands':
        return(29)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Oman':
        return(30)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Other':
        return(31)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Pakistan':
        return(32)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Panama':
        return(33)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Portugal':
        return(34)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Romania':
        return(35)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Singapore':
        return(36)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'South Africa':
        return(37)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'South Korea':
        return(38)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Spain':
        return(39)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Sri Lanka':
        return(40)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Sweden':
        return(41)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Thailand':
        return(42)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Turkey':
        return(43)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'United Arab Em':
        return(44)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'United Kingdom':
        return(45)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Unknown':
        return(46)
    elif row['COUNTRY.OF.ORIGIN_TRIM'] == 'Vietnam':
        return(47)
    else:
        return(46)

# Assign Class Labels
validation_df['LABELS'] = validation_df.apply(class_labeler, axis=1)

### Class Labels Need to Be Numeric Based on Updated Information _TRIM VAR
validation_df['LABELS_FINAL'] = validation_df['LABELS'].astype('category')

### Xs are Featured Engineered Above and y is Numeric Category Codes from Country 
y_valid = validation_df['LABELS_FINAL']

In [52]:
### Expand the StopWords List by Eyeballing things that don't make sense
stop_words = text.ENGLISH_STOP_WORDS.union(['item','vendor','type','po','dept','description',
                                            'pls','cargo','assortment','container','contains',
                                            'material','materials','shipment','shipper','qty',
                                            'purchase','product','procedures','pcs','order',
                                            'destination','delivery','count','code','ct',
                                            'department','negotiable','negotiated','non',
                                            'number','packaging','supplier'])

corpus = validation_df['PRODUCT.DETAILS'].values.astype('U')
cv = CountVectorizer(max_features = 100, stop_words=stop_words)
product_details_parsed = cv.fit_transform(corpus)
print(cv.get_feature_names())

['00', '00007', '00014', '00016', '00018', '00025', '00063', '0020', '0023', '0040', '0042', '0043', '0073', '0078742000008', '10', '100', '12', '13', '16', '18', '20', '2013', '24', '30', '40', '43', '48', '5oz', '9503', '9503000073', 'artificial', 'asst', 'black', 'canada', 'carrier', 'cartons', 'cbm', 'certify', 'chair', 'collect', 'com', 'consistent', 'copy', 'cotton', 'ctn', 'ctns', 'cy', 'declaration', 'declare', 'details', 'elwood', 'expeditors', 'fg', 'flow', 'footwear', 'freight', 'fresh', 'furniture', 'gln', 'houston', 'hs', 'hts', 'issue', 'itemized', 'kgs', 'knit', 'ladies', 'load', 'ms', 'nan', 'op', 'packing', 'pam', 'place', 'plastic', 'polyester', 'pono', 'regulated', 'release', 'sea', 'set', 'sk', 'solid', 'southgate', 'statesboro', 'steel', 'stock', 'storag', 'storage', 'style', 'suffolk', 'table', 'toys', 'value', 'vancouver', 'virtual', 'wal', 'way', 'white', 'wood']


In [53]:
### Turn Product Details into a DF and 
df_nlp_valid = pd.DataFrame(product_details_parsed.todense(), columns=cv.get_feature_names())

### Update X to include the NLP Vectorized Features
X1_valid = pd.concat([weight_kg_valid,port_dummies_valid,dow_dummies_valid,weektype_dummies_valid,month_dummies_valid,season_dummies_valid], axis=1).reset_index()
X_valid = pd.concat([X1_valid,df_nlp_valid], axis=1)

In [54]:
#In X Valid
X_valid.columns.difference(X_train.columns)

Index(['18', 'Lawrence, Massachusetts',
       'Los Angeles International Airport, Los Angeles, California',
       'Newark, New Jersey', 'nan', 'statesboro', 'storag', 'table'],
      dtype='object')

In [55]:
### Drop Out Columns from X_valid not in X_train

del X_valid['18']
del X_valid['Lawrence, Massachusetts']
del X_valid['Los Angeles International Airport, Los Angeles, California']
del X_valid['Newark, New Jersey']
del X_valid['nan']
del X_valid['statesboro']
del X_valid['storag']
del X_valid['table']

In [56]:
#In X_Train
X_train.columns.difference(X_valid.columns)

Index(['2012', 'BOSTON', 'Brunswick, Georgia', 'CHARLESTON',
       'Carquinez Strait, California', 'Fernandina Beach, Florida',
       'Freeport, Texas', 'Houston Intercontinental Airport, Houston, Texas',
       'Morgan City, Louisiana', 'New Haven, Connecticut',
       'Newport News, Virginia', 'OAKLAND', 'OAKLAND, CA',
       'Perth Amboy, New Jersey', 'Rochester, New York',
       'Vancouver, Washington', 'allowed', 'bx', 'great', 'savannah'],
      dtype='object')

In [57]:
### Include Blank Columns where Columns in X_Train but not X_Valid

X_valid['2012'] = 0
X_valid['BOSTON'] = 0
X_valid['Brunswick, Georgia'] = 0
X_valid['CHARLESTON'] = 0
X_valid['Carquinez Strait, California'] = 0
X_valid['Fernandina Beach, Florida'] = 0
X_valid['Freeport, Texas'] = 0
X_valid['Houston Intercontinental Airport, Houston, Texas'] = 0
X_valid['Morgan City, Louisiana'] = 0
X_valid['New Haven, Connecticut'] = 0
X_valid['Newport News, Virginia'] = 0
X_valid['OAKLAND'] = 0
X_valid['OAKLAND, CA'] = 0
X_valid['Perth Amboy, New Jersey'] = 0
X_valid['Rochester, New York'] = 0
X_valid['Vancouver, Washington'] = 0
X_valid['allowed'] = 0
X_valid['bx'] = 0
X_valid['great'] = 0
X_valid['savannah'] = 0

In [60]:
### Use Model 4 to Predict on X

### Fit XGboost

from sklearn import datasets
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Fit Model 
model = XGBClassifier() 
model.fit(X_train, y_train)

### Predict on Validation Set (X_Valid) Using Model 4 Training
y_pred = model.predict(X_valid) 

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_valid, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:33:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 49.93%


In [68]:
country_key_values

['Argentina',
 'Australia',
 'Bahamas',
 'Belgium',
 'Brazil',
 'Canada',
 'Chile',
 'China',
 'China Taiwan',
 'Colombia',
 'Costa Rica',
 'Dominican Republic',
 'Egypt',
 'France',
 'Germany',
 'Guatemala',
 'Honduras',
 'Hong Kong',
 'India',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Malaysia',
 'Mexico',
 'Netherlands',
 'Oman',
 'Other',
 'Pakistan',
 'Panama',
 'Portugal',
 'Singapore',
 'South Africa',
 'South Korea',
 'Spain',
 'Sri Lanka',
 'Sweden',
 'Thailand',
 'Turkey',
 'United Arab Em',
 'United Kingdom',
 nan,
 'Unknown',
 'Vietnam']

In [67]:
from sklearn.metrics import classification_report

#Get the Labels in the Report
df1 = validation_df.sort_values('LABELS_FINAL', ascending=True).drop_duplicates('COUNTRY.OF.ORIGIN_TRIM').sort_index()
country_key = df1[['LABELS_FINAL','COUNTRY.OF.ORIGIN_TRIM']].sort_values('LABELS_FINAL', ascending=True)
country_key_values = country_key['COUNTRY.OF.ORIGIN_TRIM'].tolist()

print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00        27
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        18
           6       0.00      0.00      0.00        62
           7       0.00      0.00      0.00       648
           8       0.75      0.66      0.70     34416
           9       0.01      0.01      0.01       630
          10       0.00      0.00      0.00        26
          11       0.00      0.00      0.00       745
          12       0.00      0.02      0.00        49
          13       0.92      0.57      0.70       254
          14       0.00      0.00      0.00         9
          15       0.00      0.00      0.00        11
          17       0.01      0.00      0.00      1108
          19       0.00      0.00      0.00       145
          20       0.18    

/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamessoley/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Something clearly isn't quite right here. My model which performed very well on the 1/3rd holdout .92% accuracy in Model 4 isn't performing well at all on the validation set. A number of things could have gone wrong: 1) I may have messed up a class label strategy but I checked that a few times 2) I may have severly overfit the data but my understanding is that Xgboost does alot of work under the hood to protect against this (L1 & L2 regularization, max depth, num trees) so I'll be surprised if this is happening 3) The Validation set is very different than the training set which is always something that can be a problem and I need to think about how to reduce the error that is presented here. I've hit the 3+ hours at this point so I'll stop and leave it as a matter of discussion for the interview